In [ ]:
%run funcs.ipynb
%run Proj_org.ipynb

In [ ]:
%run funcs.ipynb

In [ ]:
skl.__version__

In [ ]:
#patch_sklearn(global_patch=True)

# Classification

In [ ]:
dt = DecisionTreeClassifier()

In [ ]:
X_train_copy, y_train_copy = pd.read_csv("X_train.csv"), pd.read_csv("y_train.csv")

In [ ]:
X_test_c, y_tests_c = pd.read_csv("X_test.csv"), pd.read_csv("y_test.csv")

In [ ]:
len(y_train_copy["ConvertedCompYearly"].unique())

In [ ]:
len(y_tests_c["ConvertedCompYearly"].unique())

In [ ]:
X_train_copy

In [ ]:
X_train_copy

In [ ]:
idx = y_train_copy.value_counts()[y_train_copy.value_counts() < 300].index

In [ ]:
idx = list(map(lambda x : x[0], idx))

In [ ]:
classes = {}
for i in idx:
    classes[i] = 350


In [ ]:
sm = SMOTE(sampling_strategy=classes ,random_state=42)

In [ ]:
X_train_copy, y_train_copy = sm.fit_resample(X_train_copy, y_train_copy)

In [ ]:
X_train_copy.to_csv("X_train_copy.csv")
y_train_copy.to_csv("y_train_copy.csv")

In [ ]:
X_t_train, X_val, y_t_train, y_val = train_test_split(X_train_copy, y_train_copy, test_size=0.2, stratify=y_train_copy)

In [ ]:
y_train_copy.value_counts()

In [ ]:
param_search = {
    "criterion" : ["gini", "entropy"],
    "max_depth" : np.append(np.arange(1, 500, 50), None),
    "min_samples_split" : np.arange(2, 200, 20),
    "min_samples_leaf" : np.arange(1, 200, 20),
    "max_leaf_nodes" : np.append(np.arange(5, 200, 20), None)
}

In [ ]:
bycs = BayesSearchCV(dt, param_search, n_iter = 40, n_jobs = -1)

In [ ]:
start = time.time()
count = 0

bycs.fit(X_t_train, y_t_train)

end = time.time()
print(end - start)

In [ ]:
dt_model = bycs.best_estimator_

In [ ]:
start = time.time()

dt_model.fit(X_t_train, y_t_train)

end = time.time()
print(end - start)

In [ ]:
dt_model.fit(X_t_train, pd.Categorical(y_t_train["ConvertedCompYearly"]).codes)

In [ ]:
#dt_model = convert(dt_model, "pytorch")
#dt_model.to("cuda")

In [ ]:
start = time.time()

preds_dt = dt_model.predict(X_val)

end = time.time()
print(end - start)

In [ ]:
start = time.time()

preds_dt_t = dt_model.predict(X_t_train)

end = time.time()
print(end - start)

In [ ]:
start = time.time()

preds_dt_c = dt_model.predict(X_test_c)

end = time.time()
print(end - start)

In [ ]:
accuracy_score(y_t_train, preds_dt_t)

In [ ]:
len(preds_dt)

In [ ]:
len(pd.Categorical(y_val["ConvertedCompYearly"]).codes)

In [ ]:
accuracy_score(pd.Categorical(y_val["ConvertedCompYearly"]).codes, preds_dt)

In [ ]:
accuracy_score(y_tests_c, preds_dt_c)

In [ ]:
cm = pd.DataFrame(confusion_matrix(y_val, preds_dt))

In [ ]:
f1_score(y_tests_c, preds_dt_c, average="weighted")

In [ ]:
y_tests_c

In [ ]:
preds_dt

In [ ]:
y_val.to_numpy().reshape(-1)

In [ ]:
len(y_val["ConvertedCompYearly"].unique())

In [ ]:
len(y_tests_c["ConvertedCompYearly"].unique())

In [ ]:
roc_auc_score(y_tests_c.to_numpy().reshape(-1), dt_model.predict_proba(X_test_c), multi_class='ovo')

In [ ]:
roc_auc_score(y_val.to_numpy().reshape(-1), dt_model.predict_proba(X_val), multi_class='ovr')

In [ ]:
cm.to_csv("confusion_matrix_T.csv")

In [ ]:
param_search_svm = {
    "C" : np.linspace(1, 3, 6),
    "kernel" : ["linear", "poly", "rbf", "sigmoid"],
    "degree" : np.arange(3, 22, 3),
    "probability" : [True],
}

In [ ]:
svm = SVC(C=param_search_svm["C"][0], kernel=param_search_svm["kernel"][0], degree=param_search_svm["degree"][0], probability=True)

In [ ]:
X_t_train[:2]

In [ ]:
start = time.time()


svm.fit(X_t_train[:2000], y_t_train[:2000])

end = time.time()
print(end - start)

In [ ]:
preds_svm = svm.predict(X_t_train)

In [ ]:
accuracy_score(y_t_train, preds_svm)

In [ ]:
preds_svm = svm.predict(X_test_c)

In [ ]:
accuracy_score(y_tests_c, preds_svm)

In [ ]:
f1_score(y_tests_c, preds_svm, average="weighted")

In [ ]:
roc_auc_score()

In [ ]:
roc_auc_score(y_tests_c.to_numpy().reshape(-1), svm.predict_proba(X_test_c), multi_class='ovo')

In [ ]:
param_search = {
    "n_neighbors" : np.arange(5, 50, 10),
    "weights" : ["uniform", "distance"],
    "leaf_size" : np.arange(30, 100, 20),
    "metric" : [distance.cosine, distance.correlation, distance.chebyshev, distance_metrics()["euclidean"]]
}

In [ ]:
knn = KNeighborsClassifier(n_jobs=-1)

In [ ]:
knn.fit(X_t_train, y_t_train)

In [ ]:
np.arange(0,10)

In [ ]:
X_val.iloc[np.arange(0,100)]

In [ ]:
preds_knn = knn.predict(X_t_train.iloc[:])

In [ ]:
accuracy_score(preds_knn, y_t_train.iloc[:])

In [ ]:
f1_score(y_tests_c, preds_knn, average="weighted")

In [ ]:
roc_auc_score(y_tests_c.to_numpy().reshape(-1), knn.predict_proba(X_test_c), multi_class='ovo')

In [ ]:
preds_knn = knn.predict(X_t_train.iloc[:])

In [ ]:
accuracy_score( y_t_train.iloc[:], preds_knn)

In [ ]:
mlpc = MLPClassifier()

In [ ]:
len(y_val.unique())

In [ ]:
mlpc.fit(X_t_train.to_numpy(), y_t_train.to_numpy().reshape(-1))

In [ ]:
preds_mlpc = mlpc.predict(X_val.to_numpy())

In [ ]:
preds_mlpc = mlpc.predict(X_t_train.to_numpy())

In [ ]:
preds_mlpc

In [ ]:
accuracy_score(y_val.to_numpy().reshape(-1), preds_mlpc)

In [ ]:
accuracy_score(y_t_train.to_numpy().reshape(-1), preds_mlpc)

In [ ]:
f1_score(y_val.to_numpy().reshape(-1), preds_mlpc, average="weighted")

In [ ]:
roc_auc_score(y_val.to_numpy().reshape(-1), mlpc.predict_proba(X_val), multi_class='ovo')

In [ ]:
accuracy_score(y_val.to_numpy().reshape(-1), preds_mlpc)

In [ ]:
X_t_train

In [ ]:
len(y_t_train["ConvertedCompYearly"].unique())


In [ ]:
y_t_train["ConvertedCompYearly"]

In [ ]:
y = pd.Categorical(y_t_train["ConvertedCompYearly"]).codes

In [ ]:
inputs = layers.Input(shape=(386,))
residual = inputs
x = layers.Dense(386, activation="relu")(inputs)
x = layers.Add()([residual, x])
out = layers.Dense(113, activation=tf.nn.softmax)(x)

In [ ]:
model = Model(inputs=inputs, outputs=out, name="keras_model")

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0003), loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_t_train, y, epochs=1000, steps_per_epoch=200, workers=-1)

In [ ]:
model.save("keras_model")

In [ ]:
y_t = model.predict(X_t_train)

In [ ]:
y_v = model.predict(X_val)

In [ ]:
y_tst = model.predict(X_test_c)

In [ ]:
tf.argmax(y_v, axis=1).numpy()

In [ ]:
pd.Categorical(y_val["ConvertedCompYearly"]).codes

In [ ]:
accuracy_score(pd.Categorical(y_t_train["ConvertedCompYearly"]).codes, tf.argmax(y_t, axis=1).numpy())

In [ ]:
accuracy_score(pd.Categorical(y_val["ConvertedCompYearly"]).codes, tf.argmax(y_v, axis=1).numpy())

In [ ]:
accuracy_score(pd.Categorical(y_tests_c["ConvertedCompYearly"]).codes, tf.argmax(y_tst, axis=1).numpy())

In [ ]:
f1_score(y_tests_c.to_numpy().reshape(-1), tf.argmax(y_tst, axis=1).numpy(), average="weighted")

In [ ]:
roc_auc_score(y_tests_c.to_numpy().reshape(-1), y_tst, multi_class='ovo')

# Regression

In [ ]:
%run funcs.ipynb

In [ ]:
X_train_com_r, X_test_com_r, y_train_com_r, y_test_com_r = \
pd.read_csv("X_train_r.csv"), pd.read_csv("X_test_r.csv"), pd.read_csv("y_train_r.csv"), pd.read_csv("y_test_r.csv"),

In [ ]:
X_t_train_r, X_val_r, y_t_train_r, y_val_r  = train_test_split(X_train_com_r, y_train_com_r, test_size=0.2)

In [ ]:
dtR = DecisionTreeRegressor()

In [ ]:
dtR.fit(X_t_train_r, y_t_train_r)

In [ ]:
preds_dtR = dtR.predict(X_test_com_r)

In [ ]:
preds_dtR

In [ ]:
y_val_r

In [ ]:
preds_dtR

In [ ]:
y_val_r.to_numpy().reshape(-1)

In [ ]:
preds_dtR = dtR.predict(X_t_train_r)

In [ ]:
preds_dtR = dtR.predict(X_t_train_r)

In [ ]:
r2_score(y_t_train_r, preds_dtR)

In [ ]:
preds_dtR = dtR.predict(X_val_r)

In [ ]:
r2_score(y_val_r, preds_dtR)

In [ ]:
preds_dtR = dtR.predict(X_test_com_r)

In [ ]:
r2_score(y_test_com_r, preds_dtR)

In [ ]:
mean_squared_error(y_test_com_r.to_numpy().reshape(-1), preds_dtR)

In [ ]:
y_val_r.to_numpy().reshape(-1)

In [ ]:
plt.Figure(dpi=200)
sns.scatterplot(x=range(len(y_val_r)), y=y_val_r.to_numpy().reshape(-1))
sns.lineplot(x=range(len(y_val_r)), y=preds_dtR, color="r")
plt.show()

In [ ]:
lr = LinearRegression()

In [ ]:
X_t_train_r.values

In [ ]:
y_t_train_r.values.reshape(-1)

In [ ]:
lr.fit(X_t_train_r.values, y_t_train_r.values.reshape(-1))

In [ ]:
preds_lR = lr.predict(X_t_train_r.values)

In [ ]:
r2_score(y_t_train_r, preds_lR)

In [ ]:
preds_lR = lr.predict(X_val_r.values)

In [ ]:
r2_score(y_val_r, preds_lR)

In [ ]:
preds_lR = lr.predict(X_test_com_r.values)

In [ ]:
r2_score(y_test_com_r, preds_lR)

In [ ]:
mean_squared_error(y_test_com_r, preds_lR)

In [ ]:
y_val_r.to_numpy().reshape(-1)

In [ ]:
preds_lR

In [ ]:
plt.Figure(dpi=200)
sns.scatterplot(x=range(len(y_t_train_r)), y=y_t_train_r.to_numpy().reshape(-1))
plt.plot(range(len(y_t_train_r)),preds_lR.reshape(-1), color="r")
plt.show()

In [ ]:
plt.Figure(dpi=200)
sns.scatterplot(x=range(len(y_val_r)), y=y_val_r.to_numpy().reshape(-1))
plt.plot(range(len(y_val_r)),preds_lR.reshape(-1), color="r")
plt.show()

In [ ]:
plt.Figure(dpi=200)
sns.scatterplot(x=range(len(y_test_com_r)), y=y_test_com_r.to_numpy().reshape(-1))
plt.plot(range(len(y_test_com_r)),preds_lR.reshape(-1), color="r")
plt.show()

In [ ]:
svr = SVR(kernel="linear", C=1.4)

In [ ]:
X_t_train_r.to_numpy()[0:100]

In [ ]:
x = X_t_train_r.to_numpy()[0:150]

In [ ]:
y = y_t_train_r.to_numpy().reshape(-1)[0:150]

In [ ]:
svr.fit(x, y)

In [ ]:
preds_svr = svr.predict(X_t_train_r.to_numpy())

In [ ]:
r2_score(y_t_train_r, preds_svr)

In [ ]:
preds_svr = svr.predict(X_val_r.to_numpy())

In [ ]:
r2_score(y_val_r, preds_svr)

In [ ]:
preds_svr = svr.predict(X_test_com_r.to_numpy())

In [ ]:
r2_score(y_test_com_r, preds_svr)

In [ ]:
mean_squared_error(y_test_com_r, preds_svr)

mlpr = MLPRegressor((200, 100, 50, 10, 50, 100, 200))

In [ ]:
mlpr = MLPRegressor((200, 100, 50, 10, 50, 100, 200))

In [ ]:
mlpr.fit(X_t_train_r, y_t_train_r)

In [ ]:
mlr = mlpr.predict(X_t_train_r)

In [ ]:
r2_score(y_t_train_r, mlr)

In [ ]:
mlr = mlpr.predict(X_val_r)

In [ ]:
r2_score(y_val_r, mlr)

In [ ]:
mlr = mlpr.predict(X_test_com_r)

In [ ]:
r2_score(y_test_com_r, mlr)

In [ ]:
mean_squared_error(y_test_com_r, mlr)

In [ ]:
#mlpr.score(X_train_r, y_train_r)

# Clustering

In [ ]:
km = KMeans(n_clusters=10)

In [ ]:
X_t_train = pd.read_csv("X_train")

In [ ]:
km.fit(X_t_train)

In [ ]:
sses = []
for i in range(1,20):
    km = KMeans(n_clusters=i)
    km.fit(X_t_train)
    sses.append(km.inertia_)
    print(i)
    


In [ ]:
sses = []
for i in range(19,40):
    km = KMeans(n_clusters=i)
    km.fit(X_t_train)
    sses.append(km.inertia_)
    print(i)

In [ ]:
sses = []
for i in range(39,80):
    km = KMeans(n_clusters=i)
    km.fit(X_t_train)
    sses.append(km.inertia_)
    print(i)

In [ ]:
sses = []
for i in range(79,200):
    km = KMeans(n_clusters=i)
    km.fit(X_t_train)
    sses.append(km.inertia_)
    print(i)

In [ ]:
sses = []
for i in range(259,300):
    km = KMeans(n_clusters=i)
    km.fit(X_t_train)
    sses.append(km.inertia_)
    print(i)

In [ ]:
len(sses)

In [ ]:
np.argmin(sses)

In [ ]:
sses

In [ ]:
km.inertia_

In [ ]:
km.inertia_

In [ ]:
lofm = LocalOutlierFactor()
lof = LocalOutlierFactor(novelty=True)

In [ ]:
lof.fit(X_t_train_r)

In [ ]:
lof.fit(y_val_r)

In [ ]:
preds = lof.predict(y_val_r)

In [ ]:
pd.Series(preds).value_counts()/len(preds)

In [ ]:
y_val_r.iloc[(preds == 1)]

In [ ]:
lof.score_samples(X_val_r)

# Plots

In [ ]:
plt.figure(dpi=250, figsize=(40,40))
plot_tree(dt_model)
plt.show()

In [ ]:
plt.figure(dpi=250, figsize=(40,40))
plot_tree(dtR)
plt.show()